In [1]:
library(devtools)
devtools::install_github("prodriguezsosa/conText")

Loading required package: usethis




cli          (3.6.0      -> 3.6.1     ) [CRAN]
rlang        (1.0.6      -> 1.1.0     ) [CRAN]
vctrs        (0.5.2      -> 0.6.2     ) [CRAN]
tibble       (3.1.8      -> 3.2.1     ) [CRAN]
pillar       (1.8.1      -> 1.9.0     ) [CRAN]
RcppArmad... (0.12.0.1.0 -> 0.12.2.0.0) [CRAN]
gtable       (0.3.1      -> 0.3.3     ) [CRAN]
dplyr        (1.1.0      -> 1.1.2     ) [CRAN]
SnowballC    (0.7.0      -> 0.7.1     ) [CRAN]
ggplot2      (3.4.1      -> 3.4.2     ) [CRAN]
quanteda     (3.2.4      -> 3.3.0     ) [CRAN]


Installing 11 packages: cli, rlang, vctrs, tibble, pillar, RcppArmadillo, gtable, dplyr, SnowballC, ggplot2, quanteda

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘cli’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘rlang’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘vctrs’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘gtable’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘pillar’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘tibble’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘dplyr’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘ggplot2’ had non-zero exit status”


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmpk2gsvh/remotese4810a1e0/prodriguezsosa-conText-523f736/DESCRIPTION’ ... OK
* preparing ‘conText’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘conText_1.4.3.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library(conText)
library(quanteda)
library(dplyr)
library(text2vec)
library(hunspell)
library(tidyverse)
# library(magrittr)

Package version: 3.3.0
Unicode version: 13.0
ICU version: 66.1

Parallel computing: 4 of 4 threads used.

See https://quanteda.io for tutorials and examples.


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘hunspell’


The following object is masked from ‘package:quanteda’:

    dictionary


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.1     ✔ stringr   1.5.0
✔ lubridate 1.9.2     ✔ tibble    3.1.8
✔ purrr     1.0.1     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [3]:
data <- readRDS("/kaggle/input/twitter-data-top-500-university/tweet_all_blm_test_30Mar23.rds")
data <- data[(data$UniSerial <= 300) & (data$MainHandle == 1), ]

In [4]:
data$AfterBLM <- "No"
data$AfterBLM[data$year <= 2018 ] <- "No"
data$AfterBLM[data$year > 2018 ] <- "Yes"

data$IsTop <- "No"
data$IsTop[data$UniSerial <= 100 ] <- "Yes"
data$IsTop[data$UniSerial > 100 ] <- "No"

In [5]:
data <- data %>% select(Tweet, AfterBLM)

In [6]:
print(dim(data))

[1] 3906616       2


In [7]:
# Pre processing 

# tokenize corpus removing unnecessary (i.e. semantically uninformative) elements
corp <- corpus(data, text_field = "Tweet")
gc()

toks <- tokens(corp, 
               remove_punct=T, 
               remove_symbols=T, 
               remove_numbers=T, 
               remove_separators=T, 
               remove_url=T, 
               verbose=F)
gc()

toks_nostop <- tokens_select(toks, 
                             pattern = c(stopwords("en")),
                             selection = "remove", 
                             min_nchar=3)
gc()

# only use features that appear at least 5 times in the corpus
feats <- dfm(toks_nostop, 
             tolower=T, 
             verbose = FALSE) %>% dfm_trim(min_termfreq = 5) %>% featnames()
gc()

# leave the pads so that non-adjacent words will not become adjacent
toks_nostop_feats <- tokens_select(toks_nostop, 
                                   feats, 
                                   padding = TRUE)
saveRDS(toks_nostop_feats, "/kaggle/working/tweet_thesis_toks_nostop_feats.rds")
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10877560,581.0,21163952,1130.3,19535268,1043.3
Vcells,101668584,775.7,308387142,2352.9,481604024,3674.4


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,18435311,984.6,42319900,2260.2,42319900,2260.2
Vcells,146633664,1118.8,1274299056,9722.2,1106731395,8443.7


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,22342570,1193.3,42319900,2260.2,42319900,2260.2
Vcells,184232486,1405.6,1019439245,7777.8,1106731395,8443.7


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,22354528,1193.9,42319900,2260.2,42319900,2260.2
Vcells,184469786,1407.4,815551396,6222.2,1106731395,8443.7


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,26261185,1402.5,42319900,2260.2,42319900,2260.2
Vcells,218621801,1668.0,652441117,4977.8,1106731395,8443.7


In [8]:
# target <- c("black")

# target_toks <- tokens_context(x = toks_nostop_feats, 
#                               pattern = target, 
#                               window = 6L)

In [9]:
# load pretrained models 

glove_model = readRDS("/kaggle/input/glovepretrained-embrgrsn/local_glove.rds")
local_transform <- readRDS("/kaggle/input/glovepretrained-embrgrsn/local_transform.rds")

In [10]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "black" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.5289023 0.008072018 0.5163721 0.5414778       0


In [11]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "racism" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate  std.error lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.5618386 0.03055347 0.509335 0.6106385       0


In [12]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "attend" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.4552799 0.007714829 0.4426484 0.4688825       0


In [13]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "race" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate  std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.7107921 0.01278017 0.6899795 0.7327323       0


In [14]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "enjoy" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.4646031 0.007949275 0.4512033 0.4774993       0


In [15]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "excited" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.3500096 0.006116172 0.3399169 0.3598332       0


In [16]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "help" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.5067119 0.003564473 0.500988 0.5125067       0


In [17]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "awesome" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate  std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.4494959 0.01194239 0.4302122 0.4685051       0


In [18]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "announce" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.5495461 0.0172307 0.5209419 0.5763316       0


In [19]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "semester" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.4585569 0.005934537 0.4490794 0.4680185       0


In [20]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "students" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.4883355 0.002115496 0.4848936 0.4917148       0


In [21]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "summer" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.6431493 0.005577958 0.6334115 0.6520963       0


In [22]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "welcome" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.4238054 0.004800217 0.4158675 0.4318415       0


In [23]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "winter" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.6218128 0.009821684 0.6060056 0.6390239       0


In [24]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "season" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate  std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.7161867 0.00779535 0.7021077 0.7288333       0


In [25]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "spring" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate   std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.6904242 0.006675851 0.6794002 0.7011805       0


In [26]:
# embedding regression model

set.seed(2023L)
model_target <- conText(formula = "christmas" ~ AfterBLM,
                  data = toks_nostop_feats,
                  pre_trained = glove_model,
                  transform = TRUE, 
                  transform_matrix = local_transform,
                  bootstrap = TRUE, 
                  num_bootstraps = 500,
                  confidence_level = 0.95, 
                  stratify = TRUE,
                  permute = TRUE, 
                  num_permutations = 500,
                  window = 6, 
                  valuetype = 'fixed',
                  case_insensitive = TRUE,
                  hard_cut = FALSE,
                  verbose = FALSE)

   coefficient normed.estimate  std.error  lower.ci  upper.ci p.value
1 AfterBLM_Yes       0.7389745 0.01775215 0.7090828 0.7677156       0
